In [1]:
import gunpowder as gp
import numpy as np
import zarr
import matplotlib.pyplot as plt
import math

In [2]:
def imshowPair(img1, img2):
    fig, ax = plt.subplots(1, 2)
    ax[0].imshow(img1)
    ax[1].imshow(img2)

def imshow(image):
  if len(image.shape) == 2:
    plt.imshow(image)
  elif len(image.shape) == 3:
    plt.imshow(image.transpose(1, 2, 0))
  else:
    fig, axes = plt.subplots(1, image.shape[0], figsize=(10, 10), sharex=True, sharey=True)
    for i, im in enumerate(image):
      axes[i].imshow(im.transpose(1, 2, 0))
    plt.show()

In [3]:
pathZarr = "/mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/dataB.zarr"
data = zarr.open(pathZarr, mode='r')
data.tree(expand=True)

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, name='gt', nodes=(Node(disabled=True, ico…

In [7]:
data[1]

KeyError: 1

In [18]:
raw = gp.ArrayKey('RAW')
gt = gp.ArrayKey('GT')

source = gp.ZarrSource(
    pathZarr,
    {
      raw: 'tr',
      gt: 'gt'
    },
    {
      raw: gp.ArraySpec(interpolatable=True),
      gt: gp.ArraySpec(interpolatable=False)
    })

request = gp.BatchRequest()
request[raw] = gp.Roi((0, 0), (256, 256))
request[gt] = gp.Roi((0, 0), (256, 256))

pipeline = (
    source +
    gp.RandomLocation() +
    gp.SimpleAugment()
)

In [19]:
with gp.build(pipeline):
    batch = pipeline.request_batch(request)

imshowPair(batch[raw].data.squeeze()[0], batch[gt].data.squeeze()[0])

something went wrong during the setup of the pipeline, calling tear down


PipelineSetupError: Exception in ZarrSource[/mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/dataB.zarr] while calling setup()